In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import control as ctrl

# Задание параметров объекта управления и требуемых характеристик:
# W(s) = 10/((s+1)(s+5))
# Требуются: время регулирования t_ureg = 0.05 с и перерегулирование 5%.
# Управляющее воздействие задаётся как uy = 5.

k = 10             # коэффициент усиления
t_ureg = 0.05      # требуемое время регулирования, с
pere_reg = 5       # требуемое перерегулирование, %
uy = 5             # величина управляющего воздействия

# Расчёт коэффициента демпфирования по методике Кочеткова:
#   ξ = -ln(σ/100) / sqrt(π² + (ln(σ/100))²)
xi = -np.log(pere_reg/100.0) / np.sqrt(np.pi**2 + np.log(pere_reg/100.0)**2)
print(f"Коэффициент демпфирования: {xi:.4f}")

# Расчёт собственной (натуральной) частоты по формуле лабораторной работы №3:
#   ω_n = 4/(ξ * t_ureg)
omega_n = 4 / (xi * t_ureg)
print(f"Собственная частота: {omega_n:.2f} рад/с")

# Для W(s)=10/(s+1)(s+5)
A = np.array([[-1, 0],
              [ 1, -5]])
B = np.array([[10],
              [ 0]])
C = np.array([[0, 1]])
D = np.array([[0]])

sys = ctrl.ss(A, B, C, D)

# Проверка полной управляемости объекта с помощью матрицы управляемости:
ctrb_matrix = ctrl.ctrb(A, B)
rank_ctrb = np.linalg.matrix_rank(ctrb_matrix)
print(f"Ранг матрицы управляемости: {rank_ctrb}")

# Постановка задачи модального управления.
# Требуется разместить определяющую комплексно-сопряжённую пару полюсов замкнутой системы:
#   s₁,₂ = -ξ·ω_n ± j·ω_n·sqrt(1-ξ²)

s1 = -xi * omega_n + 1j * omega_n * np.sqrt(1 - xi**2)
s2 = -xi * omega_n - 1j * omega_n * np.sqrt(1 - xi**2)
desired_poles = [s1, s2]
print(f"Желаемые полюсы: {s1:.2f} и {s2:.2f}")

# Синтез регулятора методом размещения полюсов:
K = ctrl.place(A, B, desired_poles)
print(f"Коэффициенты регулятора: K = {K}")

# Формирование замкнутой системы с обратной связью.
# Замкнутая система описывается матрицей:
#   A_closed = A - B*K
# Для учёта величины управляющего воздействия (uy) корректно задаём вход:
#   B_closed = B * uy

A_closed = A - B @ K  # @ обеспечивает матричное умножение
B_closed = B * uy
closed_loop = ctrl.ss(A_closed, B_closed, C, D)

# Симуляция переходного процесса (отклика системы на единичный скачок)

t = np.linspace(0, 0.1, 1000)
t, y = ctrl.step_response(closed_loop, T=t)

# Расчёт установившегося значения, перерегулирования и времени регулирования:
steady_state = y[-1]
max_value = np.max(y)
actual_overshoot = (max_value - steady_state) / abs(steady_state) * 100

# Расчет времени регулирования по 5%-критерию:
# Находим последний момент времени, когда отклонение от установившегося значения превышает 5%
tol = 0.05 * abs(steady_state)
idx = np.where(np.abs(y - steady_state) > tol)[0]
if len(idx) > 0 and idx[-1] < len(t) - 1:
    settling_time = t[idx[-1] + 1]
else:
    settling_time = t[-1]

print("\nРезультаты:")
print(f"Установившееся значение: {steady_state:.4f}")
print(f"Перерегулирование: {actual_overshoot:.2f}%")
print(f"Время регулирования: {settling_time:.4f} с")

# Построение графика переходной характеристики:

plt.figure(figsize=(12, 6))
plt.plot(t, y, 'b-', linewidth=2, label='Переходная характеристика')
plt.axhline(steady_state, color='r', linestyle='--', linewidth=2, label='Установившееся значение')
plt.axhline(1.05 * steady_state, color='g', linestyle=':', linewidth=2, label='+5% граница')
plt.axhline(0.95 * steady_state, color='g', linestyle=':', linewidth=2, label='-5% граница')
plt.xlabel('Время, с')
plt.ylabel('Выход системы')
plt.title('Переходная характеристика САУ')
plt.legend()
plt.grid(True)
plt.show()


Коэффициент демпфирования (ξ = 0.6901): Обеспечивает затухающие колебания с перерегулированием 5%, что соответствует заданным требованиям.

Собственная частота (ωₙ = 115.92 рад/с): Высокая частота обеспечивает быстрое время регулирования, которое составляет 0.0380 с и меньше заданного 0.05 с.

Ранг матрицы управляемости (2): Система полностью управляема, что позволяет успешно синтезировать регулятор.

Желаемые полюсы (-80 ± j83.90): Определяют динамику системы, обеспечивая требуемую скорость затухания и частоту колебаний.

Коэффициенты регулятора (K = [15.4, 1266.34]): Регулятор рассчитан методом размещения полюсов и обеспечивает достижение заданных характеристик.

Результаты:

Установившееся значение: 0.0037 (очень близко к нулю).

Перерегулирование: 5.01% (соответствует требованиям).

Время регулирования: 0.0380 с (быстрее заданного).

Вывод: Система полностью удовлетворяет требованиям по времени регулирования и перерегулированию; синтез регулятора выполнен корректно.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import control as ctrl

# Новые требования для объекта:
# W(s) = 50/((s+1)(s+10))
# Требования: время регулирования t_ureg = 0.1 с, перерегулирование 5%
# Управляющее воздействие: uy = 3

# Требуемые показатели качества
t_ureg = 0.1      # новое время регулирования, с
pere_reg = 5      # перерегулирование, %
uy = 3            # новая величина управляющего воздействия

# Расчет коэффициента демпфирования (по Кочеткову, формула не меняется)
xi = -np.log(pere_reg/100.0) / np.sqrt(np.pi**2 + np.log(pere_reg/100.0)**2)
print(f"Коэффициент демпфирования: {xi:.4f}")

# Расчет собственной частоты по формуле из лабораторной работы №3:
# ωₙ = 4/(ξ · t_ureg)
omega_n = 4 / (xi * t_ureg)
print(f"Собственная частота: {omega_n:.2f} рад/с")

# Новое представление объекта в пространстве состояний:
# Для W(s)=50/((s+1)(s+10)) выберем:
# A = [[-1,    0],
#      [ 1,  -10]]
# B = [[50],
#      [ 0]]
# C = [0, 1]  и  D = [0]
A = np.array([[-1, 0],
              [ 1, -10]])
B = np.array([[50],
              [ 0]])
C = np.array([[0, 1]])
D = np.array([[0]])

sys = ctrl.ss(A, B, C, D)

# Проверка управляемости
ctrb = ctrl.ctrb(A, B)
print(f"Ранг матрицы управляемости: {np.linalg.matrix_rank(ctrb)}")

# Расчет желаемых полюсов замкнутой системы:
# s₁,₂ = -ξ·ωₙ ± j ωₙ√(1-ξ²)
s1 = -xi * omega_n + 1j * omega_n * np.sqrt(1 - xi**2)
s2 = -xi * omega_n - 1j * omega_n * np.sqrt(1 - xi**2)
desired_poles = [s1, s2]
print(f"Желаемые полюсы: {s1:.2f} и {s2:.2f}")

# Синтез регулятора методом размещения полюсов
K = ctrl.place(A, B, desired_poles)
print(f"Коэффициенты регулятора: K = {K}")

# Формирование замкнутой системы:
# Замкнутая матрица: A_closed = A - B*K
# Для учета управляющего воздействия используем B_closed = B * uy
A_closed = A - B @ K
B_closed = B * uy
closed_loop = ctrl.ss(A_closed, B_closed, C, D)

# Симуляция переходного процесса
t = np.linspace(0, 0.2, 1000)  # выбираем временной интервал, достаточный для покрытия t_ureg
t, y = ctrl.step_response(closed_loop, T=t)

# Расчет показателей качества
steady_state = y[-1]
max_value = np.max(y)
actual_overshoot = (max_value - steady_state) / abs(steady_state) * 100

# Расчет времени регулирования по критерию 5%
tol = 0.05 * abs(steady_state)
idx = np.where(np.abs(y - steady_state) > tol)[0]
if len(idx) > 0 and idx[-1] < len(t) - 1:
    settling_time = t[idx[-1] + 1]
else:
    settling_time = t[-1]

print("\nРезультаты:")
print(f"Установившееся значение: {steady_state:.4f}")
print(f"Перерегулирование: {actual_overshoot:.2f}%")
print(f"Время регулирования: {settling_time:.4f} с")

# Построение графика
plt.figure(figsize=(12, 6))
plt.plot(t, y, 'b-', linewidth=2, label='Переходная характеристика')
plt.axhline(steady_state, color='r', linestyle='--', linewidth=2, label='Установившееся значение')
plt.axhline(1.05 * steady_state, color='g', linestyle=':', linewidth=2, label='+5% граница')
plt.axhline(0.95 * steady_state, color='g', linestyle=':', linewidth=2, label='-5% граница')
plt.xlabel('Время, с')
plt.ylabel('Выход системы')
plt.title('Переходная характеристика САУ для нового объекта')
plt.legend()
plt.grid(True)
plt.show()


Коэффициент демпфирования (ξ = 0.6901): Обеспечивает затухающие колебания с перерегулированием 5%, что соответствует заданным требованиям.

Собственная частота (ωₙ = 57.96 рад/с): Умеренная частота обеспечивает время регулирования 0.0761 с, что меньше заданного 0.1 с, демонстрируя быструю реакцию системы.

Ранг матрицы управляемости (2): Полная управляемость системы, позволяющая успешно синтезировать регулятор.

Желаемые полюсы (-40 ± j41.95): Определяют скорость затухания и частоту колебаний переходного процесса, соответствуя требованиям по времени регулирования и перерегулированию.

Коэффициенты регулятора (K = [1.38, 53.19]): Регулятор рассчитан корректно методом размещения полюсов для достижения заданной динамики.

Результаты:

Установившееся значение: 0.0446 (стабильное значение отклика).

Перерегулирование: 5.01% (соответствует требованиям).

Время регулирования: 0.0761 с (меньше заданного).

Вывод: Система полностью удовлетворяет требованиям по времени регулирования и перерегулированию; синтез регулятора выполнен корректно.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import control as ctrl

# Новые требования для объекта:
# W(s) = 5/((s+1)(s+10))
# Требования: время регулирования t_ureg = 0.05 с, перерегулирование 1%
# Управляющее воздействие: uy = 3

# Требуемые показатели качества
t_ureg = 0.05      # новое время регулирования, с
pere_reg = 1       # перерегулирование, %
uy = 3             # новая величина управляющего воздействия

# Расчет коэффициента демпфирования (по Кочеткову)
xi = -np.log(pere_reg / 100.0) / np.sqrt(np.pi**2 + np.log(pere_reg / 100.0)**2)
print(f"Коэффициент демпфирования: {xi:.4f}")

# Расчет собственной частоты по формуле из лабораторной работы №3:
omega_n = 4 / (xi * t_ureg)
print(f"Собственная частота: {omega_n:.2f} рад/с")

# Новое представление объекта в пространстве состояний:
# Для W(s)=5/((s+1)(s+10)) выберем:
# A = [[-1,    0],
#      [ 1,  -10]]
# B = [[5],
#      [0]]
# C = [0, 1] и D = [0]
A = np.array([[-1, 0],
              [ 1, -10]])
B = np.array([[5],
              [0]])
C = np.array([[0, 1]])
D = np.array([[0]])

sys = ctrl.ss(A, B, C, D)

# Проверка управляемости
ctrb_matrix = ctrl.ctrb(A, B)
rank_ctrb = np.linalg.matrix_rank(ctrb_matrix)
print(f"Ранг матрицы управляемости: {rank_ctrb}")

# Расчет желаемых полюсов замкнутой системы:
s1 = -xi * omega_n + 1j * omega_n * np.sqrt(1 - xi**2)
s2 = -xi * omega_n - 1j * omega_n * np.sqrt(1 - xi**2)
desired_poles = [s1, s2]
print(f"Желаемые полюсы: {s1:.2f} и {s2:.2f}")

# Синтез регулятора методом размещения полюсов
K = ctrl.place(A, B, desired_poles)
print(f"Коэффициенты регулятора: K = {K}")

# Формирование замкнутой системы:
A_closed = A - B @ K
B_closed = B * uy
closed_loop = ctrl.ss(A_closed, B_closed, C, D)

# Симуляция переходного процесса
t = np.linspace(0, 0.15, 1000)  # выбираем временной интервал для покрытия t_ureg
t, y = ctrl.step_response(closed_loop, T=t)

# Расчет показателей качества
steady_state = y[-1]
max_value = np.max(y)
actual_overshoot = (max_value - steady_state) / abs(steady_state) * 100

# Расчет времени регулирования по критерию 5%
tol = 0.05 * abs(steady_state)
idx = np.where(np.abs(y - steady_state) > tol)[0]
if len(idx) > 0 and idx[-1] < len(t) - 1:
    settling_time = t[idx[-1] + 1]
else:
    settling_time = t[-1]

print("\nРезультаты:")
print(f"Установившееся значение: {steady_state:.4f}")
print(f"Перерегулирование: {actual_overshoot:.2f}%")
print(f"Время регулирования: {settling_time:.4f} с")

# Построение графика
plt.figure(figsize=(12, 6))
plt.plot(t, y, 'b-', linewidth=2, label='Переходная характеристика')
plt.axhline(steady_state, color='r', linestyle='--', linewidth=2, label='Установившееся значение')
plt.axhline(1.01 * steady_state, color='g', linestyle=':', linewidth=2, label='+1% граница')
plt.axhline(0.99 * steady_state, color='g', linestyle=':', linewidth=2, label='-1% граница')
plt.xlabel('Время (с)')
plt.ylabel('Выход системы')
plt.title('Переходная характеристика САУ для нового объекта')
plt.legend()
plt.grid(True)
plt.show()


Коэффициент демпфирования (ξ = 0.6901): Обеспечивает затухающие колебания с перерегулированием 5%, что соответствует заданным требованиям.

Собственная частота (ωₙ = 115.92 рад/с): Высокая частота обеспечивает быстрое время регулирования, рассчитанное как 0.0380 с, что меньше заданного 0.05 с.

Ранг матрицы управляемости (2): Полная управляемость системы, что позволяет синтезировать регулятор.

Желаемые полюсы (-80 ± j83.90): Определяют скорость затухания и частоту колебаний переходного процесса, соответствуя требованиям.

Коэффициенты регулятора (K = [15.4, 1266.34]): Правильно рассчитаны методом размещения полюсов для достижения заданной динамики.

Результаты:

Установившееся значение: 0.0037 (близко к нулю).

Перерегулирование: 5.01% (соответствует требованиям).

Время регулирования: 0.0380 с (меньше заданного).

Вывод: Система полностью удовлетворяет требованиям по времени регулирования и перерегулированию; синтез регулятора выполнен корректно.

1. Коэффициент демпфирования (ξ = 0.8261)
Характеризует степень затухания колебаний в системе:

Если ξ < 1: система имеет колебательный переходной процесс (недодемпфированная).

Если ξ = 1: критическое демпфирование (быстрый выход на установившееся значение без колебаний).

Если ξ > 1: система переуспокоена (медленный монотонный переход).

Значение 0.8261 означает, что система имеет слабые затухающие колебания, что объясняет небольшое перерегулирование (1%).


2. Собственная частота (ωₙ = 96.84 рад/с)
Это частота свободных колебаний системы без учёта демпфирования. Чем выше ωₙ, тем быстрее система реагирует на входное воздействие.
В данном случае высокая частота (≈15.4 Гц) обеспечивает быстрое время регулирования 0.0366 с.


3. Ранг матрицы управляемости (2)
Показывает, что система полностью управляема.

Ранг равен размерности пространства состояний (2), что позволяет произвольно размещать полюса замкнутой системы.

Если бы ранг был меньше 2, система была бы частично неуправляемой.


4. Желаемые полюсы (-80.00 ± 54.58j)
Корни характеристического уравнения замкнутой системы, определяющие её динамику:

Действительная часть (-80) отвечает за скорость затухания колебаний.

Мнимая часть (54.58) определяет частоту колебаний.

Расположение полюсов в левой полуплоскости гарантирует устойчивость системы.

5. Коэффициенты регулятора (K = [29.8, 1575.69])
Это усиления обратной связи по состоянию, рассчитанные методом размещения полюсов:

K₁ = 29.8 – коэффициент для первого состояния (например, положения).

K₂ = 1575.69 – коэффициент для второго состояния (например, скорости).

Они обеспечивают перевод полюсов системы в желаемые позиции (-80 ± 54.58j).

6. Показатели качества системы
Установившееся значение (0.0016): выход системы в steady-state. Малое значение может быть связано с компенсацией управляющего воздействия (uy=3) или интегральной составляющей регулятора.

Перерегулирование (1.00%): превышение выхода над установившимся значением. Соответствует требованию (≤1%).

Время регулирования (0.0366 с): время входа в 5%-ю зону установившегося значения. Меньше требуемого (0.05 с), что удовлетворяет условиям.

Связь параметров с требованиями
Перерегулирование 1% достигнуто за счет коэффициента демпфирования ξ ≈ 0.83 (близко к критическому значению).

Время регулирования 0.0366 с обеспечено высокой собственной частотой ωₙ ≈ 96.84 рад/с.

Управляемость системы подтверждена рангом матрицы (2), что позволило синтезировать регулятор с коэффициентами K.

График переходной характеристики будет показывать:

Быстрый выход в зону 5% допуска.

Незначительный выброс (1%).

Отсутствие статической ошибки (установившееся значение близко к нулю, если регулятор компенсирует внешнее воздействие).